In [12]:
import skimage
import os
import pandas as pd
import numpy as np
from IPython.display import display
import matplotlib
import matplotlib.pyplot as plt
from skimage import data
from skimage import io
import glob
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import LassoCV
from sklearn.preprocessing import PolynomialFeatures
import re
from scipy import stats
from skimage.viewer import ImageViewer
import cv2
from sklearn.metrics import r2_score
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture
from sklearn.externals import joblib
import subprocess
import time
import statsmodels.regression.linear_model as sm
from statsmodels.api import OLS



In [13]:
from RecSystemKM import RecSystemKM_test

In [14]:

path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df = pd.read_csv(path+'users_followers_merged.csv')

### pull a sample for the pipeline
df = df[2:4]

# Scrape pics from users - don't rerun

for i, v in enumerate(df.username): 
    time.sleep(20)
    subprocess.check_output(["instaloader", (str(v))])
    

# Create the Engagment pipeline

In [15]:


temp_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'
train_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'
save_path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'
dest_img =  '/Users/kimia/Desktop/Capstone/hotel/Rec_system/User_pics/'

userlist= ['or_shmul','fathy_abo_dahis', 'iamkurdapya', 'abhishekraghav015',  'jnbax']
#user list will be the picture number! 


# Vectorize images 

In [16]:
c = RecSystemKM_test(train_path, save_path )
c.Karans_get_train_images(userlist)


#Create Object

train_df = c.convert_to_features(columns = ["User_Handle","URL","R_Mean", "R_STD", "R_MED", "G_Mean", "G_STD", 
                "G_MED", "B_Mean", "B_STD", "B_MED", "Canny", "ORB_X", "ORB_Y"])

Number of images loaded: 298


In [25]:
# now merge with # of followers

In [17]:
path = '/Users/kimia/Desktop/Capstone/hotel/Rec_system/'
df_usernames = pd.read_csv(path+'users_followers_merged.csv')
df_usernames['User_Handle'] = df_usernames.username

#df_usernames['followers'] = (df_usernames['followers'].apply(lambda x: x.replace('k',"000").replace(".","").replace("m","000000").replace(",","")))
#df_usernames.followers = pd.to_numeric(df_usernames.followers)

In [18]:
%%file Engagement_pipeline.py

from sklearn import linear_model

class Engagement:
    
    #Initialization
    def __init__(self, train_path, save_path):
        self.train_path = train_path
        self.save_path = save_path
        
    def zscore(x,mu,std):
        zscore = (x-mu)/std
        return(zscore)

    def metrics(self, df): 
        df[['likes_count','followers']] = df[['likes_count','followers']].apply(pd.to_numeric)

        df['likes_score'] = (df['likes_count'])/(df['followers'])

        df['comments_score'] = (df['comment_count'])/(df['followers'])

        df['zscore_likes'] = zscore(df['likes_score'], df.likes_score.std(), df.likes_score.mean())

        df['zscore_comments'] = zscore(df['comments_score'], df.comments_score.std(), df.comments_score.mean())

        df['final_score'] = df['zscore_likes'] + df['zscore_comments']
        
        return df
        
    def prep_model(self, df, columns_to_remove): 
    
        df = df.drop(columns_to_remove, axis = 1, inplace = True)
        
        rands = np.random.seed(9001)
        msk = np.random.rand(len(df)) < 0.25
        data_train = df[~msk]
        data_test = df[msk]
        
        y_train = data_train.final_score
        y_test = data_test.final_score

        predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
               'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
        X_train = data_train[predictors]
        X_test = data_test[predictors]
        
        return y_train, y_text, X_train, X_test ### should these be self?
        
    def linear(): ## yes those should be self if this is to run automatically
        
        lm = linear_model.LinearRegression() 
        model = lm.fit(X_train,y_train) 
        predictions = lm.predict(X_train)

        model.score(X_train,y_train), model.score(X_test,y_test)
        
    def Ridge():
        alpha= [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2,10**3, ]
        ridge = RidgeCV(alphas=alpha, cv=10, fit_intercept=True)
        ridge.fit(X_train, y_train)
        
        train_score = ridge.score(X_train, y_train)
        test_score = ridge.score(X_test, y_test)
        return(train_score, test_score)



    def Lasso():
        alpha= [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2,10**3, ]
        lasso = LassoCV(alphas=alpha, cv=10, max_iter=10000, fit_intercept=True)
        lasso.fit(X_train, y_train)
        
        train_score = lasso.score(X_train, y_train)
        test_score = lasso.score(X_test, y_test)
        
        return(train_score, test_score)

        
        ### I also did Poly Feats but seems poor choice here

Overwriting Engagement_pipeline.py


In [19]:


def zscore(x,mu,std):
    zscore = (x-mu)/std
    return(zscore)

def metrics(df): 
    df[['likes_count','followers']] = df[['likes_count','followers']].apply(pd.to_numeric)

    df['likes_score'] = (df['likes_count'])/(df['followers'])
    
    df['comments_score'] = (df['comment_count'])/(df['followers'])
    
    df['zscore_likes'] = zscore(df['likes_score'], df.likes_score.std(), df.likes_score.mean())
    
    df['zscore_comments'] = zscore(df['comments_score'], df.comments_score.std(), df.comments_score.mean())
    
    df['final_score'] = df['zscore_likes'] + df['zscore_comments']
    

In [20]:
df_usernames = df_usernames[0:5]
metrics(df_usernames)

In [21]:
df_usernames

,Unnamed: 0,followers,username,id_post,video_count,url_img,link_post,owner,caption,comment_count,...,taken_at_time,shortcode,is_video,likes_count,User_Handle,likes_score,comments_score,zscore_likes,zscore_comments,final_score
0,0,1283.0,or_shmul,1627655237685711385,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BaWl9CjD5IZ,45547257,💞,7,...,2017-10-17 14:47:26,BaWl9CjD5IZ,False,138,or_shmul,0.107560,0.005456,1.685628,1.836331,3.521959
1,1,741.0,fathy_abo_dahis,1641992705372560728,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbJh6p3g51Y,3470341263,NaN,1,...,2017-11-06 09:33:25,BbJh6p3g51Y,False,24,fathy_abo_dahis,0.032389,0.001350,-0.204240,-0.456359,-0.660599
2,2,867.0,iamkurdapya,1644299880334917484,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbRugd_lH9s,226907530,☝,0,...,2017-11-09 13:57:22,BbRugd_lH9s,False,6,iamkurdapya,0.006920,0.000000,-0.844528,-1.209822,-2.054351
3,3,5131.0,abhishekraghav015,1637736956104215547,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/Ba6aRWgHwv7,3106737762,#like4like#followforfollow#justinbieber#selena...,9,...,2017-10-31 12:38:00,Ba6aRWgHwv7,False,211,abhishekraghav015,0.041123,0.001754,0.015337,-0.230511,-0.215174
4,4,5051.0,misstribu,1643430709473145791,0,https://instagram.fprg2-1.fna.fbcdn.net/t51.28...,https://www.instagram.com/p/BbOo4XjHXO_,595372894,#regram @nathalie_dmitrovic #newshop #marseill...,2,...,2017-11-08 09:10:29,BbOo4XjHXO_,False,55,misstribu,0.010889,0.000396,-0.744757,-0.988751,-1.733508


In [22]:
df = train_df.merge(df_usernames,how='left',on='User_Handle')

In [23]:
# drop extra columns

In [24]:
df.columns

Index(['User_Handle', 'URL', 'R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD',
       'G_MED', 'B_Mean', 'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',
       'Unnamed: 0', 'followers', 'username', 'id_post', 'video_count',
       'url_img', 'link_post', 'owner', 'caption', 'comment_count',
       'taken_at_timestamp', 'taken_at_time', 'shortcode', 'is_video',
       'likes_count', 'likes_score', 'comments_score', 'zscore_likes',
       'zscore_comments', 'final_score'],
      dtype='object')

In [26]:
remove = ['User_Handle', 'URL', 'Unnamed: 0', 'username',  'followers', 'username', 'id_post', 'video_count',
       'url_img', 'link_post', 'owner', 'caption', 'comment_count',
       'taken_at_timestamp', 'taken_at_time', 'shortcode', 'is_video',
       'likes_count', 'likes_score', 'comments_score',
       'zscore_likes', 'zscore_comments',]
df.drop(remove, axis = 1, inplace = True)

In [27]:
## any null?
print(df.isnull().sum(axis=0))

R_Mean         0
R_STD          0
R_MED          0
G_Mean         0
G_STD          0
G_MED          0
B_Mean         0
B_STD          0
B_MED          0
Canny          0
ORB_X          0
ORB_Y          0
final_score    0
dtype: int64


# build a model

In [28]:
rands = np.random.seed(9001)
msk = np.random.rand(len(df)) < 0.25
data_train = df[~msk]
data_test = df[msk]

In [29]:
y_train = data_train.final_score
y_test = data_test.final_score

predictors = ['R_Mean', 'R_STD', 'R_MED', 'G_Mean', 'G_STD', 'G_MED', 'B_Mean',
       'B_STD', 'B_MED', 'Canny', 'ORB_X', 'ORB_Y',]
X_train = data_train[predictors]
X_test = data_test[predictors]

In [31]:
len(y_train), len(X_train)

(220, 220)

In [32]:
for i, v in enumerate(X_train.columns): 
    column = v
    X_train[str(column)] = X_train[str(column)].astype(float)
    


/anaconda/envs/python3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [33]:
y_train = y_train.astype(float)

In [34]:


X_train_constant= sm.add_constant(X_train)
X_test_constant= sm.add_constant(X_test)

#MODEL = sm.OLS(y_train.astype(float), X_train_constant.astype(float)).fit()
#results = model_train.fit()

model = sm.OLS(np.asarray((y_train)), np.asarray((X_train))).fit()
predictions = model.predict(X_train)



In [35]:
from sklearn import linear_model
lm = linear_model.LinearRegression() 
model = lm.fit(X_train,y_train) 
predictions = lm.predict(X_train)

model.score(X_train,y_train), model.score(X_test,y_test)

(0.1439929957948708, -0.04782148068336611)

In [36]:
alpha= [10**-5, 10**-4, 10**-3, 10**-2, 10**-1, 10**0, 10**1, 10**2,10**3, ]
ridge = RidgeCV(alphas=alpha, cv=10, fit_intercept=True)
ridge.fit(X_train, y_train)
print(ridge.fit(X_train, y_train))
ridge.score(X_train, y_train), ridge.score(X_test, y_test)


RidgeCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000], cv=10,
    fit_intercept=True, gcv_mode=None, normalize=False, scoring=None,
    store_cv_values=False)


(0.1414840209879603, 0.0010873433083060657)

In [37]:
ridge.score(X_train, y_train), ridge.score(X_test, y_test)

(0.1414840209879603, 0.0010873433083060657)

In [38]:


##
lasso = LassoCV(alphas=alpha, cv=10, max_iter=10000, fit_intercept=True)
print(lasso.fit(X_train, y_train))


LassoCV(alphas=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100, 1000],
    copy_X=True, cv=10, eps=0.001, fit_intercept=True, max_iter=10000,
    n_alphas=100, n_jobs=1, normalize=False, positive=False,
    precompute='auto', random_state=None, selection='cyclic', tol=0.0001,
    verbose=False)


In [39]:
lasso.score(X_train, y_train), lasso.score(X_test, y_test)

(0.0, -0.0010378470514764704)

In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LinearRegression

for i,v in enumerate([1,2,3,4,5]): 
    
    a = PolynomialFeatures(v)
    modelo2 = make_pipeline(a, LinearRegression())
    model_for_list2 = modelo2.fit(X_train, y_train)
    r2 = r2_score(y_train, model_for_list2.predict(X_train))
    r2_test = r2_score(y_test, model_for_list2.predict(X_test))
    print(v, "train : ", r2)
    print(v, "test : ", r2_test)


1 train :  0.1439929957948709
1 test :  -0.04782148068336656
2 train :  -0.11880588420509652
2 test :  -8.299663933745897
3 train :  1.0
3 test :  -31.64246490528741
4 train :  1.0
4 test :  -15.206364321824921
5 train :  1.0
5 test :  -18.637328030884962


In [47]:
for i,v in enumerate([1,2,3,4,5]):     
    a = PolynomialFeatures(v)
    modelo2 = make_pipeline(a, LassoCV())
    model_for_list2 = modelo2.fit(X_train, y_train)
    r2 = r2_score(y_train, model_for_list2.predict(X_train))
    r2_test = r2_score(y_test, model_for_list2.predict(X_test))
    print(v, "train : ", r2)
    print(v, "test : ", r2_test)


1 train :  0.0
1 test :  -0.0010378470514764704


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


2 train :  0.0
2 test :  -0.0010378470514764704


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


3 train :  0.0
3 test :  -0.0010378470514764704


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


4 train :  0.0
4 test :  -0.0010378470514764704


/anaconda/envs/python3/lib/python3.6/site-packages/sklearn/linear_model/coordinate_descent.py:491: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


5 train :  0.0
5 test :  -0.0010378470514764704


In [48]:
for i,v in enumerate([1,2,3,4,5]):     
    a = PolynomialFeatures(v)
    modelo2 = make_pipeline(a, RidgeCV())
    model_for_list2 = modelo2.fit(X_train, y_train)
    r2 = r2_score(y_train, model_for_list2.predict(X_train))
    r2_test = r2_score(y_test, model_for_list2.predict(X_test))
    print(v, "train : ", r2)
    print(v, "test : ", r2_test)


1 train :  0.14399236121864867
1 test :  -0.04694548071255933
2 train :  0.6605831449633021
2 test :  -1.375255658576748
3 train :  0.9999999999999707
3 test :  -32.228028702078106
4 train :  0.9999999999999981
4 test :  -15.111097773767387
5 train :  0.9999999999999999
5 test :  -18.726973353675746
